In [68]:
# load the required packages
from google.cloud import storage
import os
import fnmatch
from datetime import datetime

In [3]:
# load the credentials file
storage_client = storage.Client.from_service_account_json("chodrine.json")

# define the bucket name
bucket_name = "ocular"
bucket = storage_client.get_bucket(bucket_name)
exclude_extensions = ['.jpeg', '.jpg', '.png', '.json']

In [4]:
# load folders and sub-folders
blobs = bucket.list_blobs()
for blob in blobs:
    if exclude_extensions and any(blob.name.endswith(ext) for ext in exclude_extensions):
        continue
    print(f"Folder: {blob.name}")

Folder: Cervical-Cancer/
Folder: Malaria/
Folder: Malaria/Data-Collection/
Folder: Malaria/Data-Collection/Mulago-Kirudu/
Folder: Tuberculosis/


All images have to be uploaded to a new sub-folder and this sub-folder will have a name as the current date and time. This will be automatically generated

In [5]:
# get current date and time
current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H:%M:%S")
formatted_datetime

'2023-11-13_16:50:59'

In [57]:
image_paths = []
def list_files_with_extension(start_path, extension):
    for root, dirs, files in os.walk(start_path):
        for file in fnmatch.filter(files, f'*.{extension}'):
            file_path = os.path.join(root, file)
            image_paths.append(file_path)
            print(file_path)

In [58]:
# Specify the directory path and file extension
directory_path = '/Users/chodry/chodrine/notebooks/Ocular/images'
file_extension = 'jpg'

In [59]:
images_to_upload = list_files_with_extension(directory_path, file_extension)

/Users/chodry/chodrine/notebooks/Ocular/images/cassava.jpg
/Users/chodry/chodrine/notebooks/Ocular/images/validate/cassava-6.jpg
/Users/chodry/chodrine/notebooks/Ocular/images/test/cassava-2.jpg
/Users/chodry/chodrine/notebooks/Ocular/images/test/day-one/cassava-3.jpg
/Users/chodry/chodrine/notebooks/Ocular/images/test/day-two/cassava-4.jpg
/Users/chodry/chodrine/notebooks/Ocular/images/train/cassava-5.jpg


In [64]:
# image_paths
image_names = [os.path.basename(file_path) for file_path in image_paths]
image_names
# len(image_names)

['cassava.jpg',
 'cassava-6.jpg',
 'cassava-2.jpg',
 'cassava-3.jpg',
 'cassava-4.jpg',
 'cassava-5.jpg']

In [67]:
# upload images to the bucket

# first we specify the folder to upload the images  (check the forth cell for the available folders)
# the folder below is an example but make sure the last folder name is the current date (formatted_datetime)
# This applies to all uploads

folder = f'Malaria/Data-Collection/Makerere/{formatted_datetime}'

for image_path in image_paths:
    image_name = os.path.basename(image_path)
    blob = bucket.blob(f'Malaria/Data-Collection/Makerere/{formatted_datetime}/{image_name}')
    with open(image_path, 'rb') as image:
        blob.upload_from_file(image, content_type='image/*')